## Cleaning Data


In [11]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error



In [12]:
# Cargar el archivo CSV
df = pd.read_csv('bbdd/productos_celulares.csv', sep=';')  # Ajusta el delimitador si es necesario

# Supongamos que df es tu DataFrame
df.columns = df.columns.str.lower()


Se debe realizar una limpieza de la base de de datos, en particular la variables tiendas y precios. 

In [13]:
# 1. Dividir la columna 'tiendas_y_precios' en múltiples columnas usando ';' como delimitador
tiendas_precios_split = df['tiendas_y_precios'].str.split(';', expand=True)

# 2. Renombrar las nuevas columnas de forma dinámica como 'tiendas_y_precios1', 'tiendas_y_precios2', etc.
tiendas_precios_split.columns = [f'tiendas_y_precios{i+1}' for i in range(tiendas_precios_split.shape[1])]

# 3. Concatenar las columnas divididas al DataFrame original
df = pd.concat([df, tiendas_precios_split], axis=1)

# 4. Definir una función para extraer tienda, precio y la presencia de la palabra "liberado"
def extraer_tienda_precio_liberado(texto):
    if isinstance(texto, str):  # Verifica si es una cadena de texto
        match = re.search(r'(.+): \$(\d+\.?\d*)', texto)
        liberado = 1 if "liberado" in texto.lower() else 0  # Verifica si contiene "liberado"
        
        if match:
            tienda = match.group(1).strip()
            precio = float(match.group(2).replace(".", ""))
            return tienda, precio, liberado
    return None, None, 0

# 5. Aplicar la función a cada columna que empieza con "tiendas_y_precios" y crear columnas 'tienda_n', 'precio_n', 'liberado_n'
for i, col in enumerate(tiendas_precios_split.columns, start=1):
    df[[f'tienda{i}', f'precio{i}', f'liberado{i}']] = df[col].apply(lambda x: pd.Series(extraer_tienda_precio_liberado(x)))

# 6. (Opcional) Eliminar las columnas intermedias 'tiendas_y_precios1', 'tiendas_y_precios2', etc.
df.drop(columns=tiendas_precios_split.columns, inplace=True)

# Mostrar el DataFrame resultante
#df.head()


El Scrapping, lamentablemente, tuvo el problema de precios en valores perdidos. 
Para modelar de mejor manera se procedió a pivotear la base de datos, es decir, transformarla de largo a ancho. 
En total se quedó en 1659 celulares ofrecidos, en términos del cruce que existe con los modelos y las marcas que lo ofrcen. 

In [14]:
# Convertir de formato ancho a largo
df_long = pd.wide_to_long(df.reset_index(), 
                          stubnames=['tienda', 'precio', 'liberado'], 
                          i='index', 
                          j='variable', 
                          sep='',
                          suffix='\d+').reset_index()


#print(df_long)


**Se procede a limpiar el resto de las variables**

Es importante intentar limpiar el texto correspondiente. 

In [15]:
df_long['camara'] = df_long['cámara_principal'].str.extract(r"(?<=Estándar )([0-9.]+)(?= MP)").astype(float)

# Extraer solo el valor numérico de la columna Batería y convertir a numérico
df_long['bateria'] = df_long['batería'].str.extract(r"(\d+)").astype(float)

# Extraer solo el valor numérico de la columna Memoria_interna y convertir a numérico
df_long['memoria'] = df_long['memoria_interna'].str.extract(r"(\d+)").astype(float)

# Extraer solo el valor numérico de la columna Potencia_de_carga y convertir a numérico
df_long['potencia'] = df_long['potencia_de_carga'].str.extract(r"(\d+)").astype(float)

In [9]:
# Eliminar todas las filas que contengan valores perdidos en cualquiera de las columnas
df_long_cleaned = df_long.dropna().reset_index(drop=True)

# Mostrar el DataFrame resultante sin valores perdidos
#print(df_long_cleaned)


Habiendo realizado lo anterior, la base de datos complementante depurada

In [10]:
df_long_cleaned.to_csv("bbdd/final2.csv", sep=";", index=False, encoding="utf-8")